<a href="https://colab.research.google.com/github/Vac1k/Aurora_Project/blob/main/Aurora_Predict_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт бібліотек

In [6]:
import pandas as pd

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document

import numpy as np

In [4]:
!pip install python-docx




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


Завантаження та обробка даних про товари

In [7]:
file_path_inventory = "/content/Залишки по ТТ.xlsx"
inventory_df = pd.read_excel(file_path_inventory)

In [8]:
inventory_df = inventory_df.rename(columns={
    "Рівень категорії 3": "Category_Level3",
    "Назва товару": "Product_Full_Name",
    "К-сть залишків на кінець дня": "Stock_Quantity_End_Day",
    "Собівартість залишків на кінець дня": "Stock_Cost_End_Day"
})

In [9]:
inventory_df["Product_ID"] = inventory_df["Product_Full_Name"].str.extract(r'^(\d+)')
inventory_df["Product_Name"] = inventory_df["Product_Full_Name"].str.replace(r'^\d+\s+', '', regex=True)
inventory_df["Product_ID"] = pd.to_numeric(inventory_df["Product_ID"], errors='coerce')
inventory_df["Calculated_Price"] = (inventory_df["Stock_Cost_End_Day"] / inventory_df["Stock_Quantity_End_Day"]).round(3)

Завантаження подій та їх аналіз

In [10]:
def load_events_from_docx():
    doc = Document("/content/12123.docx")
    events_dict, current_event = {}, None

    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if not text:
            continue
        if text[0].isdigit() and "." in text:
            current_event = text.split(". ", 1)[1]
            events_dict[current_event] = []
        elif current_event:
            events_dict[current_event].append(text)
    return events_dict

events_data = load_events_from_docx()

Векторизація товарів для AI-пошуку

In [11]:
ai_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
inventory_df["Product_Vector"] = inventory_df["Product_Name"].apply(lambda x: ai_model.encode(x))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Фільтрація товарів за подією

In [25]:


def filter_by_event(event_name):
    allowed_keywords = events_data.get(event_name, [])
    if not allowed_keywords:
        return pd.DataFrame(columns=inventory_df.columns)

    filtered_df = inventory_df[inventory_df["Product_Name"].apply(
        lambda x: any(word.lower() in x.lower() for word in allowed_keywords)
    )]

    return filtered_df.head(2)

AI-пошук товарів

In [36]:
def find_similar_products(query, top_n=5):
    query_vector = np.array(ai_model.encode(query)).reshape(1, -1)
    product_vectors = np.array(list(inventory_df["Product_Vector"]))

    similarity_scores = cosine_similarity(query_vector, product_vectors).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    return inventory_df.iloc[top_indices][["Product_Name", "Stock_Quantity_End_Day"]].head(top_n)

Speech Recognition

In [27]:
def recognize_speech():
    recognizer = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            audio = recognizer.listen(source)
            return recognizer.recognize_google(audio, language="uk-UA")
    except (sr.UnknownValueError, sr.RequestError, OSError):
        return None

Основна логіка програми

In [37]:
def process_user_query(user_input):
    user_input = user_input.lower().strip()
    matched_event = next((event for event in events_data if event.lower() in user_input), None)

    if matched_event:
        return filter_by_event(matched_event)

    # Просто використовуємо оригінальні слова запиту без розширення
    return find_similar_products(user_input, top_n=2)

In [38]:
def main():
    mode = input("🔹 Виберіть режим (1 - текстовий ввод, 2 - голосовий ввод): ").strip()
    user_query = input("✍ Введіть ваш запит: ") if mode == "1" else recognize_speech()
    if not user_query:
        return

    recommendations = process_user_query(user_query)
    print(f"📌 Рекомендації для запиту '{user_query}':")
    print(recommendations.to_string(index=False))

In [44]:
main()

🔹 Виберіть режим (1 - текстовий ввод, 2 - голосовий ввод): 1
✍ Введіть ваш запит: хочу
📌 Рекомендації для запиту 'хочу':
                            Product_Name  Stock_Quantity_End_Day
Крем дит. Домашній Доктор Збір трав,42мл                      10
Крем дит. Домашній Доктор Календула,42мл                       8
